In [1]:
# Primero definimos algunas funciones auxiliares que nos ayudarán a convertir de un número a una lista de bits y viceversa.
function original_integer(list)
    return parse(Int, join(list); base=2)
end

function base_2(integer; pad= nothing)
    if pad == nothing
        return reverse(digits(integer, base = 2))
    else
        return reverse(digits(integer, base = 2, pad = pad))
    end
end

base_2 (generic function with 1 method)

In [3]:
A = base_2(5; pad=nothing)

3-element Vector{Int64}:
 1
 0
 1

In [47]:
original_integer([0,1,1,1,0])

14

In [99]:
function rotate(index,N)
    vec = base_2(index, pad=N)
    return original_integer([vec[end];vec[1:end-1]])
end

rotate (generic function with 2 methods)

In [100]:
rotate(5,8)

130